## References
- input references here

## Load Library

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()

## Load & Clean Data

In [36]:
# training data
train1 = pd.read_csv('data/train_FD001.txt', sep=' ', header=None)
train2 = pd.read_csv('data/train_FD002.txt', sep=' ', header=None)
train3 = pd.read_csv('data/train_FD003.txt', sep=' ', header=None)
train4 = pd.read_csv('data/train_FD004.txt', sep=' ', header=None)
# testing data
test1 = pd.read_csv('data/test_FD001.txt', sep=' ', header=None)
test2 = pd.read_csv('data/test_FD002.txt', sep=' ', header=None)
test3 = pd.read_csv('data/test_FD003.txt', sep=' ', header=None)
test4 = pd.read_csv('data/test_FD004.txt', sep=' ', header=None)
# remaining life
RUL1 = pd.read_csv('data/RUL_FD001.txt', sep=' ', header=None)
RUL2 = pd.read_csv('data/RUL_FD002.txt', sep=' ', header=None)
RUL3 = pd.read_csv('data/RUL_FD003.txt', sep=' ', header=None)
RUL4 = pd.read_csv('data/RUL_FD004.txt', sep=' ', header=None)

In [67]:
# data dimensions
pd.DataFrame(np.array([ [ np.shape(train1)[0], np.shape(train2)[0], np.shape(train3)[0], np.shape(train4)[0], np.shape(test1)[0], np.shape(test2)[0], np.shape(test3)[0], np.shape(test4)[0], np.shape(RUL1)[0], np.shape(RUL2)[0], np.shape(RUL3)[0], np.shape(RUL4)[0] ], 
                        [ np.shape(train1)[1], np.shape(train2)[1], np.shape(train3)[1], np.shape(train4)[1], np.shape(test1)[1], np.shape(test2)[1], np.shape(test3)[1], np.shape(test4)[1], np.shape(RUL1)[1], np.shape(RUL2)[1], np.shape(RUL3)[1], np.shape(RUL4)[1] ] ]), 
             index=['#columns', '#rows'],
             columns=['train1', 'train2', 'train3', 'train4', 'test1', 'test2', 'test3', 'test4', 'URL1', 'URL2', 'URL3', 'URL4'])

,train1,train2,train3,train4,test1,test2,test3,test4,URL1,URL2,URL3,URL4
#columns,20631,53759,24720,61249,13096,33991,16596,41214,100,259,100,248
#rows,28,28,28,28,28,28,28,28,2,2,2,2


In [71]:
# Add header
header = ['id','cycle','setting1','setting2','setting3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9',
          'sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21', '','']
train1.columns = header
train2.columns = header
train3.columns = header
train4.columns = header
test1.columns = header
test2.columns = header
test3.columns = header
test4.columns = header

In [73]:
train1[:5]

,id,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,,
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN


In [74]:
test1[:5]

,id,cycle,setting1,setting2,setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,,
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,NaN,NaN
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,NaN,NaN
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,NaN,NaN
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,NaN,NaN
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,NaN,NaN
